In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Necessary Libraries
import numpy as np
import pandas as pd
import recordlinkage

In [3]:
# Read customer_pivot_monthly dataset
customer_monthly_df = pd.read_csv('customer_pivot_monthly.csv')
customer_monthly_df.head()

,customer_id,customer_fiscal_start_date,customer_fiscal_start_year_month,customer_fiscal_start_year_quarter,customer_fiscal_start_month,customer_fiscal_start_quarter,customer_fiscal_start_year,customer_calendar_start_date,customer_calendar_start_year_month,customer_calendar_start_year_quarter,...,revenue_total_2020-12-01,revenue_total_2021-01-01,revenue_total_2021-02-01,revenue_total_2021-03-01,revenue_total_2021-04-01,revenue_total_2021-05-01,revenue_total_2021-06-01,revenue_total_2021-07-01,revenue_total_2021-08-01,revenue_total_2021-09-01
0,4F6BFADE-3F05-456D-8BC9-B630282DAF4A,25-08-2016,2016-FM8,2016-FQ3,FM8,FQ3,FY2016,25-11-2015,2015-CM11,2015-CM4,...,4.643,3.621,3.488,3.375,3.354,4.569,4.515,4.577,4.543,4.456
1,0EA3FD69-9F1E-4E33-A9A4-CD3E53150323,17-09-2013,2013-FM9,2013-FQ3,FM9,FQ3,FY2013,17-12-2012,2012-CM12,2012-CM4,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,D3C1E848-4000-446D-8E1D-7DEAD13360ED,31-01-2016,2016-FM1,2016-FQ1,FM1,FQ1,FY2016,30-04-2015,2015-CM4,2015-CM2,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,B9B941F4-6AF9-4FAC-A307-0BB3DC960790,10-12-2014,2014-FM12,2014-FQ4,FM12,FQ4,FY2014,10-03-2014,2014-CM3,2014-CM1,...,978.778,1214.104,206.227,339.468,614.431,219.119,1093.995,377.737,1990.203,923.632
4,D58DBC13-8DD3-4B84-8A96-7EFF40C03175,04-01-2015,2015-FM1,2015-FQ1,FM1,FQ1,FY2015,04-04-2014,2014-CM4,2014-CM2,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [4]:
customer_monthly_df.shape

(340784, 200)

The dataset consists of 340784 rows and 200 columns.

In [5]:
# Check unique customer_id
customer_monthly_df['customer_id'].nunique()

340784

Customer monthly dataset consists of 340784 unique customer_id.

In [6]:
# Keep relevant columns in customer_df dataframe from customer_monthly_df
customer_df = customer_monthly_df[['customer_id','customer_fiscal_start_date','customer_last_txn_fiscal_date',
                                  's_customer_region_grouped']]
customer_df.head()

,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped
0,4F6BFADE-3F05-456D-8BC9-B630282DAF4A,25-08-2016,23-09-2021,UK
1,0EA3FD69-9F1E-4E33-A9A4-CD3E53150323,17-09-2013,07-01-2017,UK
2,D3C1E848-4000-446D-8E1D-7DEAD13360ED,31-01-2016,18-01-2018,ANZ
3,B9B941F4-6AF9-4FAC-A307-0BB3DC960790,10-12-2014,29-09-2021,ANZ
4,D58DBC13-8DD3-4B84-8A96-7EFF40C03175,04-01-2015,07-01-2018,ANZ


In [7]:
trans_loc = customer_monthly_df.columns.get_loc(customer_monthly_df.filter(regex='transaction_total').columns[-1])
principal_loc = customer_monthly_df.columns.get_loc(customer_monthly_df.filter(regex='principal_total').columns[-1])
revenue_loc = customer_monthly_df.columns.get_loc(customer_monthly_df.filter(regex='revenue_total').columns[-1])

In [8]:
# Calculate last 12,24,36 months transaction count, principal and revenue
customer_df['L12M Transaction Count'] = customer_monthly_df.iloc[:,trans_loc-11:trans_loc+1].sum(axis=1)
customer_df['L24M Transaction Count'] = customer_monthly_df.iloc[:,trans_loc-23:trans_loc+1].sum(axis=1)
customer_df['L36M Transaction Count'] = customer_monthly_df.iloc[:,trans_loc-35:trans_loc+1].sum(axis=1)

customer_df['L12M Principal'] = customer_monthly_df.iloc[:,principal_loc-11:principal_loc+1].sum(axis=1)
customer_df['L24M Principal'] = customer_monthly_df.iloc[:,principal_loc-23:principal_loc+1].sum(axis=1)
customer_df['L36M Principal'] = customer_monthly_df.iloc[:,principal_loc-36:principal_loc+1].sum(axis=1)

customer_df['L12M Revenue'] = customer_monthly_df.iloc[:,revenue_loc-11:revenue_loc+1].sum(axis=1)
customer_df['L24M Revenue'] = customer_monthly_df.iloc[:,revenue_loc-23:revenue_loc+1].sum(axis=1)
customer_df['L36M Revenue'] = customer_monthly_df.iloc[:,revenue_loc-36:revenue_loc+1].sum(axis=1)

customer_df.head()

,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped,L12M Transaction Count,L24M Transaction Count,L36M Transaction Count,L12M Principal,L24M Principal,L36M Principal,L12M Revenue,L24M Revenue,L36M Revenue
0,4F6BFADE-3F05-456D-8BC9-B630282DAF4A,25-08-2016,23-09-2021,UK,11,22,33,2805.813,5.238281e+03,7.608446e+03,44.720,82.652,116.786
1,0EA3FD69-9F1E-4E33-A9A4-CD3E53150323,17-09-2013,07-01-2017,UK,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000
2,D3C1E848-4000-446D-8E1D-7DEAD13360ED,31-01-2016,18-01-2018,ANZ,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000
3,B9B941F4-6AF9-4FAC-A307-0BB3DC960790,10-12-2014,29-09-2021,ANZ,232,409,572,7585907.442,1.172172e+07,1.442004e+07,9323.508,14236.523,17191.043
4,D58DBC13-8DD3-4B84-8A96-7EFF40C03175,04-01-2015,07-01-2018,ANZ,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000


In [9]:
customer_df.shape

(340784, 13)

In [10]:
# Read transaction_dataset
xls = pd.ExcelFile('transaction_dataset_anonymised.xlsx')
cust_forward_12m = pd.read_excel(xls,'customer_forward_12Months')
cust_spot_12m = pd.read_excel(xls,'customer_spot_12Months')

In [11]:
cust_forward_12m.head()

,Customer_Entity_Type,Customer_Sub_Region,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Deal_Count,Deal_Turnover_AUD,Deal_Revenue_AUD,Unnamed: 11,Customer_Company_Name
0,Private Company,ANZ,40934,44236,168,1.358079e+08,130910.280,168,1.358079e+08,130910.280,0,hiboNicapan(Jing)TradCodLt
1,Private Company,North America,42775,44216,14,4.847419e+07,39139.607,14,4.847419e+07,39139.607,0,xDiancePerformorthNLtd.
2,Private Company,North America,40729,44235,21,3.529216e+07,27061.061,21,3.529216e+07,27061.061,0,orthNicanAmerduceProyersBuitedLim
3,Private Company,Europe,40961,44231,20,3.411600e+07,79328.134,20,3.411600e+07,79328.134,0,DomuandsBritedLim
4,Private Company,Europe,43593,44235,41,3.257107e+07,128498.367,41,3.257107e+07,128498.367,0,FITsingLicen(uk)itedLim


In [12]:
cust_spot_12m.head()

,Customer_Entity_Type,Customer_Sub_Region,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Deal_Count,Deal_Turnover_AUD,Deal_Revenue_AUD,Unnamed: 11,Customer_Company_Name
0,Private Company,ANZ,43759,44236,445,2.064513e+08,156436.728,445,2.064513e+08,156436.728,0,rleyWirebaaliaAustryPtdLt
1,Private Company,ANZ,43913,44236,164,1.221593e+08,81462.146,164,1.221593e+08,81462.146,0,WNEMAIYPTDLT
2,Private Company,Asia,41962,44232,319,1.013317e+08,61143.446,319,1.013317e+08,61143.446,0,nhaoJuecomTelonalInternatiited(hk)lim
3,Private Company,Asia,41215,44236,782,8.364419e+07,56807.458,782,8.364419e+07,56807.458,0,BasedardStaningsHolditedLim
4,Private Company,Europe,44025,44222,17,6.155147e+07,152362.393,17,6.155147e+07,152362.393,0,ambiKicesServitedLim


In [13]:
cust_forward_12m.shape

(890, 12)

In [14]:
cust_forward_12m.isnull().sum()

Customer_Entity_Type     0
Customer_Sub_Region      0
Unnamed: 3               0
Unnamed: 4               0
Unnamed: 5               0
Unnamed: 6               0
Unnamed: 7               0
Deal_Count               0
Deal_Turnover_AUD        0
Deal_Revenue_AUD         0
Unnamed: 11              0
Customer_Company_Name    0
dtype: int64

In [15]:
#Check for duplicate Customer_Company_Name
cust_forward_12m['Customer_Company_Name'].nunique()

889

customer_forward_12Months data contains 889 unique Customer_Company_Name.

In [16]:
#Find the duplicate Customer_Company_Name
duplicate_company_name = cust_forward_12m[cust_forward_12m['Customer_Company_Name'].duplicated() == True]['Customer_Company_Name'].tolist()

In [17]:
# Remove the rows having duplicate Customer_Company_Name
cust_forward_12m = cust_forward_12m[~cust_forward_12m['Customer_Company_Name'].isin(duplicate_company_name)]
cust_forward_12m.shape

(888, 12)

In [18]:
cust_spot_12m.shape

(20506, 12)

In [19]:
cust_spot_12m.isnull().sum()

Customer_Entity_Type     0
Customer_Sub_Region      0
Unnamed: 3               0
Unnamed: 4               0
Unnamed: 5               0
Unnamed: 6               0
Unnamed: 7               0
Deal_Count               0
Deal_Turnover_AUD        0
Deal_Revenue_AUD         0
Unnamed: 11              0
Customer_Company_Name    1
dtype: int64

In [20]:
# Remove the rows having missing data in Customer_Company_Name
cust_spot_12m = cust_spot_12m[cust_spot_12m['Customer_Company_Name'].isnull() == False]
cust_spot_12m.shape

(20505, 12)

In [21]:
#Check for duplicate Customer_Company_Name
cust_spot_12m['Customer_Company_Name'].nunique()

20488

In [22]:
#Find the duplicate Customer_Company_Name
duplicate_company_name = cust_spot_12m[cust_spot_12m['Customer_Company_Name'].duplicated() == True]['Customer_Company_Name'].tolist()

In [23]:
# Remove the rows having duplicate Customer_Company_Name
cust_spot_12m = cust_spot_12m[~cust_spot_12m['Customer_Company_Name'].isin(duplicate_company_name)]
cust_spot_12m.shape

(20471, 12)

In [24]:
df_12m = pd.merge(cust_forward_12m,cust_spot_12m,on=['Customer_Company_Name','Customer_Sub_Region'],how='outer')
df_12m.shape

(20529, 22)

In [25]:
df_12m.head()

,Customer_Entity_Type_x,Customer_Sub_Region,Unnamed: 3_x,Unnamed: 4_x,Unnamed: 5_x,Unnamed: 6_x,Unnamed: 7_x,Deal_Count_x,Deal_Turnover_AUD_x,Deal_Revenue_AUD_x,...,Customer_Entity_Type_y,Unnamed: 3_y,Unnamed: 4_y,Unnamed: 5_y,Unnamed: 6_y,Unnamed: 7_y,Deal_Count_y,Deal_Turnover_AUD_y,Deal_Revenue_AUD_y,Unnamed: 11_y
0,Private Company,ANZ,40934.0,44236.0,168.0,1.358079e+08,130910.280,168.0,1.358079e+08,130910.280,...,Private Company,40934.0,44236.0,3.0,6.917842e+06,4026.185,3.0,6.917842e+06,4026.185,0.0
1,Private Company,North America,42775.0,44216.0,14.0,4.847419e+07,39139.607,14.0,4.847419e+07,39139.607,...,Private Company,42775.0,44216.0,2.0,4.839953e+06,8482.412,2.0,4.839953e+06,8482.412,0.0
2,Private Company,North America,40729.0,44235.0,21.0,3.529216e+07,27061.061,21.0,3.529216e+07,27061.061,...,Private Company,40729.0,44235.0,9.0,1.664328e+07,12037.274,9.0,1.664328e+07,12037.274,0.0
3,Private Company,Europe,40961.0,44231.0,20.0,3.411600e+07,79328.134,20.0,3.411600e+07,79328.134,...,Private Company,40961.0,44231.0,63.0,4.418549e+07,47316.736,63.0,4.418549e+07,47316.736,0.0
4,Private Company,Europe,43593.0,44235.0,41.0,3.257107e+07,128498.367,41.0,3.257107e+07,128498.367,...,Private Company,43593.0,44235.0,1763.0,5.623181e+07,90829.708,1763.0,5.623181e+07,90829.708,0.0


In [26]:
df_12m['Customer_Company_Name'].nunique()

20529

In [27]:
df_12m['L12M Transaction Count'] = df_12m['Deal_Count_x']+df_12m['Deal_Count_y']
df_12m['L12M Principal'] = df_12m['Deal_Turnover_AUD_x']+df_12m['Deal_Turnover_AUD_y']
df_12m['L12M Revenue'] = df_12m['Deal_Revenue_AUD_x']+df_12m['Deal_Revenue_AUD_y']
df_12m.head()

,Customer_Entity_Type_x,Customer_Sub_Region,Unnamed: 3_x,Unnamed: 4_x,Unnamed: 5_x,Unnamed: 6_x,Unnamed: 7_x,Deal_Count_x,Deal_Turnover_AUD_x,Deal_Revenue_AUD_x,...,Unnamed: 5_y,Unnamed: 6_y,Unnamed: 7_y,Deal_Count_y,Deal_Turnover_AUD_y,Deal_Revenue_AUD_y,Unnamed: 11_y,L12M Transaction Count,L12M Principal,L12M Revenue
0,Private Company,ANZ,40934.0,44236.0,168.0,1.358079e+08,130910.280,168.0,1.358079e+08,130910.280,...,3.0,6.917842e+06,4026.185,3.0,6.917842e+06,4026.185,0.0,171.0,1.427258e+08,134936.465
1,Private Company,North America,42775.0,44216.0,14.0,4.847419e+07,39139.607,14.0,4.847419e+07,39139.607,...,2.0,4.839953e+06,8482.412,2.0,4.839953e+06,8482.412,0.0,16.0,5.331415e+07,47622.019
2,Private Company,North America,40729.0,44235.0,21.0,3.529216e+07,27061.061,21.0,3.529216e+07,27061.061,...,9.0,1.664328e+07,12037.274,9.0,1.664328e+07,12037.274,0.0,30.0,5.193545e+07,39098.335
3,Private Company,Europe,40961.0,44231.0,20.0,3.411600e+07,79328.134,20.0,3.411600e+07,79328.134,...,63.0,4.418549e+07,47316.736,63.0,4.418549e+07,47316.736,0.0,83.0,7.830149e+07,126644.870
4,Private Company,Europe,43593.0,44235.0,41.0,3.257107e+07,128498.367,41.0,3.257107e+07,128498.367,...,1763.0,5.623181e+07,90829.708,1763.0,5.623181e+07,90829.708,0.0,1804.0,8.880289e+07,219328.075


In [28]:
df_12m = df_12m[['Customer_Company_Name','Customer_Sub_Region','L12M Transaction Count','L12M Principal', 'L12M Revenue']]
df_12m.head()

,Customer_Company_Name,Customer_Sub_Region,L12M Transaction Count,L12M Principal,L12M Revenue
0,hiboNicapan(Jing)TradCodLt,ANZ,171.0,1.427258e+08,134936.465
1,xDiancePerformorthNLtd.,North America,16.0,5.331415e+07,47622.019
2,orthNicanAmerduceProyersBuitedLim,North America,30.0,5.193545e+07,39098.335
3,DomuandsBritedLim,Europe,83.0,7.830149e+07,126644.870
4,FITsingLicen(uk)itedLim,Europe,1804.0,8.880289e+07,219328.075


In [29]:
cust_forward_24m = pd.read_excel(xls,'customer_forward_24Months')
cust_spot_24m = pd.read_excel(xls,'customer_spot_24Months')

In [30]:
cust_forward_24m.shape

(25718, 6)

In [31]:
cust_forward_24m.isnull().sum()

Customer_Entity_Type     0
Customer_Sub_Region      0
Deal_Count               0
Deal_Turnover_AUD        0
Deal_Revenue_AUD         0
Customer_Company_Name    1
dtype: int64

In [32]:
# Remove the rows having missing data in Customer_Company_Name
cust_forward_24m = cust_forward_24m[cust_forward_24m['Customer_Company_Name'].isnull() == False]
cust_forward_24m.shape

(25717, 6)

In [33]:
#Check for duplicate Customer_Company_Name
cust_forward_24m['Customer_Company_Name'].nunique()

25715

In [34]:
#Find the duplicate Customer_Company_Name
duplicate_company_name = cust_forward_24m[cust_forward_24m['Customer_Company_Name'].duplicated() == True]['Customer_Company_Name'].tolist()

In [35]:
# Remove the rows having duplicate Customer_Company_Name
cust_forward_24m = cust_forward_24m[~cust_forward_24m['Customer_Company_Name'].isin(duplicate_company_name)]
cust_forward_24m.shape

(25713, 6)

In [36]:
cust_forward_24m.head()

,Customer_Entity_Type,Customer_Sub_Region,Deal_Count,Deal_Turnover_AUD,Deal_Revenue_AUD,Customer_Company_Name
0,Private Company,ANZ,384,2.804268e+08,283308.528,hiboNicapan(Jing)TradCodLt
1,Private Company,ANZ,504,2.160684e+08,163686.518,rleyWirebaaliaAustryPtdLt
2,Private Company,Europe,4073,1.977646e+08,555993.792,FITsingLicen(uk)itedLim
3,Private Company,Asia,544,1.734044e+08,97055.024,nhaoJuecomTelonalInternatiited(hk)lim
4,Private Company,Asia,1688,1.728246e+08,166972.558,BasedardStaningsHolditedLim


In [37]:
cust_forward_24m.rename(columns = {'Deal_Count' : 'L24M Transaction Count', 'Deal_Turnover_AUD' : 'L24M Principal',
                                   'Deal_Revenue_AUD' : 'L24M Revenue'}, inplace = True)

In [38]:
cust_forward_24m = cust_forward_24m[['Customer_Company_Name','Customer_Sub_Region','L24M Transaction Count','L24M Principal','L24M Revenue']]
cust_forward_24m.head()

,Customer_Company_Name,Customer_Sub_Region,L24M Transaction Count,L24M Principal,L24M Revenue
0,hiboNicapan(Jing)TradCodLt,ANZ,384,2.804268e+08,283308.528
1,rleyWirebaaliaAustryPtdLt,ANZ,504,2.160684e+08,163686.518
2,FITsingLicen(uk)itedLim,Europe,4073,1.977646e+08,555993.792
3,nhaoJuecomTelonalInternatiited(hk)lim,Asia,544,1.734044e+08,97055.024
4,BasedardStaningsHolditedLim,Asia,1688,1.728246e+08,166972.558


In [39]:
cust_forward_36m = pd.read_excel(xls,'customer_forward_36Months')
cust_spot_36m = pd.read_excel(xls,'customer_spot_36Months')

In [40]:
cust_forward_36m.shape

(29862, 6)

In [41]:
cust_forward_36m.isnull().sum()

Customer_Entity_Type     0
Customer_Sub_Region      0
Deal_Count               0
Deal_Turnover_AUD        0
Deal_Revenue_AUD         0
Customer_Company_Name    1
dtype: int64

In [42]:
# Remove the rows having missing data in Customer_Company_Name
cust_forward_36m = cust_forward_36m[cust_forward_36m['Customer_Company_Name'].isnull() == False]
cust_forward_36m.shape

(29861, 6)

In [43]:
#Check for duplicate Customer_Company_Name
cust_forward_36m['Customer_Company_Name'].nunique()

29855

In [44]:
#Find the duplicate Customer_Company_Name
duplicate_company_name = cust_forward_36m[cust_forward_36m['Customer_Company_Name'].duplicated() == True]['Customer_Company_Name'].tolist()

In [45]:
# Remove the rows having duplicate Customer_Company_Name
cust_forward_36m = cust_forward_36m[~cust_forward_36m['Customer_Company_Name'].isin(duplicate_company_name)]
cust_forward_36m.shape

(29849, 6)

In [46]:
cust_forward_36m.rename(columns = {'Deal_Count' : 'L36M Transaction Count', 'Deal_Turnover_AUD' : 'L36M Principal',
                                   'Deal_Revenue_AUD' : 'L36M Revenue'}, inplace = True)

In [47]:
cust_forward_36m = cust_forward_36m[['Customer_Company_Name','Customer_Sub_Region','L36M Transaction Count','L36M Principal','L36M Revenue']]
cust_forward_36m.head()

,Customer_Company_Name,Customer_Sub_Region,L36M Transaction Count,L36M Principal,L36M Revenue
0,hiboNicapan(Jing)TradCodLt,ANZ,596,4.064567e+08,416901.886
1,HLOWHIGKETSMARYPTDLT,ANZ,56,2.803069e+08,306891.622
2,nhaoJuecomTelonalInternatiited(hk)lim,Asia,772,2.583026e+08,134274.741
3,BasedardStaningsHolditedLim,Asia,2296,2.469984e+08,248898.722
4,duruHeMoniitedLim,ANZ,9838,2.458509e+08,338463.415


In [48]:
cust_forward_lt = pd.read_excel(xls,'customer_forward_Lifetime')
cust_spot_lt = pd.read_excel(xls,'customer_spot_Lifetime')

In [49]:
cust_forward_lt.shape

(4381, 21)

In [50]:
cust_forward_lt.isnull().sum()

Customer_Entity_Type        0
Customer_Sub_Region         0
First                       0
Last Deal                   0
Deal_Count                  0
Deal_Turnover_AUD           0
Deal_Revenue_AUD            0
Deal_Count.1                0
Deal_Turnover_AUD.1         0
Deal_Revenue_AUD.1          0
Unnamed: 11                 0
Unnamed: 12              4381
Unnamed: 13              4381
Unnamed: 14              4381
Unnamed: 15              4381
Unnamed: 16              4375
Unnamed: 17              4375
Unnamed: 18              4375
Unnamed: 19              4375
Unnamed: 20              4375
Customer_Company_Name       0
dtype: int64

In [51]:
#Check for duplicate Customer_Company_Name
cust_forward_lt['Customer_Company_Name'].nunique()

4369

In [52]:
#Find the duplicate Customer_Company_Name
duplicate_company_name = cust_forward_lt[cust_forward_lt['Customer_Company_Name'].duplicated() == True]['Customer_Company_Name'].tolist()

In [53]:
# Remove the rows having duplicate Customer_Company_Name
cust_forward_lt = cust_forward_lt[~cust_forward_lt['Customer_Company_Name'].isin(duplicate_company_name)]
cust_forward_lt.shape

(4365, 21)

In [54]:
cust_spot_lt.isnull().sum()

Customer_Entity_Type         0
Customer_Sub_Region          0
First                        0
Last Deal                    0
Deal_Count                   0
Deal_Turnover_AUD            0
Deal_Revenue_AUD             0
Deal_Count.1                 0
Deal_Turnover_AUD.1          0
Deal_Revenue_AUD.1           0
Unnamed: 11                  0
Unnamed: 12              49105
Unnamed: 13              49105
Unnamed: 14              49105
Unnamed: 15              49105
Unnamed: 16              49099
Unnamed: 17              49099
Unnamed: 18              49099
Unnamed: 19              49099
Unnamed: 20              49099
Customer_Company_Name       10
dtype: int64

In [55]:
# Remove the rows having missing data in Customer_Company_Name
cust_spot_lt = cust_spot_lt[cust_spot_lt['Customer_Company_Name'].isnull() == False]
cust_spot_lt.isnull().sum()

Customer_Entity_Type         0
Customer_Sub_Region          0
First                        0
Last Deal                    0
Deal_Count                   0
Deal_Turnover_AUD            0
Deal_Revenue_AUD             0
Deal_Count.1                 0
Deal_Turnover_AUD.1          0
Deal_Revenue_AUD.1           0
Unnamed: 11                  0
Unnamed: 12              49095
Unnamed: 13              49095
Unnamed: 14              49095
Unnamed: 15              49095
Unnamed: 16              49089
Unnamed: 17              49089
Unnamed: 18              49089
Unnamed: 19              49089
Unnamed: 20              49089
Customer_Company_Name        0
dtype: int64

In [56]:
cust_spot_lt.shape

(49095, 21)

In [57]:
cust_spot_lt['Customer_Company_Name'].nunique()

48886

In [58]:
duplicate_company_name = cust_spot_lt[cust_spot_lt['Customer_Company_Name'].duplicated() == True]['Customer_Company_Name'].tolist()

In [59]:
cust_spot_lt = cust_spot_lt[~cust_spot_lt['Customer_Company_Name'].isin(duplicate_company_name)]

In [60]:
cust_spot_lt.shape

(48796, 21)

In [61]:
transaction_df = pd.merge(cust_forward_lt,cust_spot_lt,on=['Customer_Company_Name','First','Last Deal','Customer_Sub_Region'],how='outer')
transaction_df.shape

(48923, 38)

In [62]:
transaction_df = transaction_df[['Customer_Company_Name','Customer_Sub_Region','First','Last Deal']]
transaction_df.head()

,Customer_Company_Name,Customer_Sub_Region,First,Last Deal
0,hiboNicapan(Jing)TradCodLt,ANZ,2012-01-26,2021-02-09
1,DickmithSnicsElectroyPtdLt,ANZ,2012-11-26,2017-12-20
2,DomuandsBritedLim,Europe,2012-02-22,2021-02-04
3,orthNicanAmerduceProyersBuitedLim,North America,2011-07-05,2021-02-08
4,BasedardStaningsHolditedLim,Asia,2012-11-02,2021-02-09


In [63]:
transaction_df = pd.merge(transaction_df,df_12m,on=['Customer_Company_Name','Customer_Sub_Region'],how='outer')
transaction_df.head()

,Customer_Company_Name,Customer_Sub_Region,First,Last Deal,L12M Transaction Count,L12M Principal,L12M Revenue
0,hiboNicapan(Jing)TradCodLt,ANZ,2012-01-26,2021-02-09,171.0,1.427258e+08,134936.465
1,DickmithSnicsElectroyPtdLt,ANZ,2012-11-26,2017-12-20,NaN,NaN,NaN
2,DomuandsBritedLim,Europe,2012-02-22,2021-02-04,83.0,7.830149e+07,126644.870
3,orthNicanAmerduceProyersBuitedLim,North America,2011-07-05,2021-02-08,30.0,5.193545e+07,39098.335
4,BasedardStaningsHolditedLim,Asia,2012-11-02,2021-02-09,794.0,8.706215e+07,63354.551


In [64]:
transaction_df = pd.merge(transaction_df,cust_forward_24m,on=['Customer_Company_Name','Customer_Sub_Region'],how='outer')
transaction_df.head()

,Customer_Company_Name,Customer_Sub_Region,First,Last Deal,L12M Transaction Count,L12M Principal,L12M Revenue,L24M Transaction Count,L24M Principal,L24M Revenue
0,hiboNicapan(Jing)TradCodLt,ANZ,2012-01-26,2021-02-09,171.0,1.427258e+08,134936.465,384.0,2.804268e+08,283308.528
1,DickmithSnicsElectroyPtdLt,ANZ,2012-11-26,2017-12-20,NaN,NaN,NaN,NaN,NaN,NaN
2,DomuandsBritedLim,Europe,2012-02-22,2021-02-04,83.0,7.830149e+07,126644.870,152.0,1.151999e+08,216923.945
3,orthNicanAmerduceProyersBuitedLim,North America,2011-07-05,2021-02-08,30.0,5.193545e+07,39098.335,58.0,9.612283e+07,88937.031
4,BasedardStaningsHolditedLim,Asia,2012-11-02,2021-02-09,794.0,8.706215e+07,63354.551,1688.0,1.728246e+08,166972.558


In [65]:
transaction_df = pd.merge(transaction_df,cust_forward_36m,on=['Customer_Company_Name','Customer_Sub_Region'],how='outer')
transaction_df.head()

,Customer_Company_Name,Customer_Sub_Region,First,Last Deal,L12M Transaction Count,L12M Principal,L12M Revenue,L24M Transaction Count,L24M Principal,L24M Revenue,L36M Transaction Count,L36M Principal,L36M Revenue
0,hiboNicapan(Jing)TradCodLt,ANZ,2012-01-26,2021-02-09,171.0,1.427258e+08,134936.465,384.0,2.804268e+08,283308.528,596.0,4.064567e+08,416901.886
1,DickmithSnicsElectroyPtdLt,ANZ,2012-11-26,2017-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DomuandsBritedLim,Europe,2012-02-22,2021-02-04,83.0,7.830149e+07,126644.870,152.0,1.151999e+08,216923.945,251.0,1.749413e+08,358150.546
3,orthNicanAmerduceProyersBuitedLim,North America,2011-07-05,2021-02-08,30.0,5.193545e+07,39098.335,58.0,9.612283e+07,88937.031,90.0,1.288484e+08,136024.523
4,BasedardStaningsHolditedLim,Asia,2012-11-02,2021-02-09,794.0,8.706215e+07,63354.551,1688.0,1.728246e+08,166972.558,2296.0,2.469984e+08,248898.722


In [66]:
transaction_df.shape

(49237, 13)

In [67]:
transaction_df.isnull().sum()

Customer_Company_Name         0
Customer_Sub_Region           0
First                       314
Last Deal                   314
L12M Transaction Count    48407
L12M Principal            48407
L12M Revenue              48407
L24M Transaction Count    23524
L24M Principal            23524
L24M Revenue              23524
L36M Transaction Count    19388
L36M Principal            19388
L36M Revenue              19388
dtype: int64

In [68]:
transaction_df['Customer_Sub_Region'].value_counts()

ANZ              29897
North America    10652
Europe            7548
Asia              1140
Name: Customer_Sub_Region, dtype: int64

In [69]:
customer_df['s_customer_region_grouped'].value_counts()

ANZ                183786
North America       74862
UK                  61436
Mainland Europe     12819
Asia                 7881
Name: s_customer_region_grouped, dtype: int64

In [70]:
customer_df['s_customer_region_grouped'] = customer_df['s_customer_region_grouped'].replace(['UK','Mainland Europe'],'Europe')

In [71]:
customer_df.head()

,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped,L12M Transaction Count,L24M Transaction Count,L36M Transaction Count,L12M Principal,L24M Principal,L36M Principal,L12M Revenue,L24M Revenue,L36M Revenue
0,4F6BFADE-3F05-456D-8BC9-B630282DAF4A,25-08-2016,23-09-2021,Europe,11,22,33,2805.813,5.238281e+03,7.608446e+03,44.720,82.652,116.786
1,0EA3FD69-9F1E-4E33-A9A4-CD3E53150323,17-09-2013,07-01-2017,Europe,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000
2,D3C1E848-4000-446D-8E1D-7DEAD13360ED,31-01-2016,18-01-2018,ANZ,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000
3,B9B941F4-6AF9-4FAC-A307-0BB3DC960790,10-12-2014,29-09-2021,ANZ,232,409,572,7585907.442,1.172172e+07,1.442004e+07,9323.508,14236.523,17191.043
4,D58DBC13-8DD3-4B84-8A96-7EFF40C03175,04-01-2015,07-01-2018,ANZ,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000


In [72]:
customer_df['customer_fiscal_start_date'] = pd.to_datetime(customer_df['customer_fiscal_start_date'])
customer_df['customer_last_txn_fiscal_date'] = pd.to_datetime(customer_df['customer_last_txn_fiscal_date'])

In [73]:
customer_df.head()

,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped,L12M Transaction Count,L24M Transaction Count,L36M Transaction Count,L12M Principal,L24M Principal,L36M Principal,L12M Revenue,L24M Revenue,L36M Revenue
0,4F6BFADE-3F05-456D-8BC9-B630282DAF4A,2016-08-25,2021-09-23,Europe,11,22,33,2805.813,5.238281e+03,7.608446e+03,44.720,82.652,116.786
1,0EA3FD69-9F1E-4E33-A9A4-CD3E53150323,2013-09-17,2017-07-01,Europe,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000
2,D3C1E848-4000-446D-8E1D-7DEAD13360ED,2016-01-31,2018-01-18,ANZ,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000
3,B9B941F4-6AF9-4FAC-A307-0BB3DC960790,2014-10-12,2021-09-29,ANZ,232,409,572,7585907.442,1.172172e+07,1.442004e+07,9323.508,14236.523,17191.043
4,D58DBC13-8DD3-4B84-8A96-7EFF40C03175,2015-04-01,2018-07-01,ANZ,0,0,0,0.000,0.000000e+00,0.000000e+00,0.000,0.000,0.000


In [74]:
transaction_df.isnull().sum()

Customer_Company_Name         0
Customer_Sub_Region           0
First                       314
Last Deal                   314
L12M Transaction Count    48407
L12M Principal            48407
L12M Revenue              48407
L24M Transaction Count    23524
L24M Principal            23524
L24M Revenue              23524
L36M Transaction Count    19388
L36M Principal            19388
L36M Revenue              19388
dtype: int64

In [75]:
transaction_df = transaction_df[~transaction_df['First'].isnull()]
transaction_df.shape

(48923, 13)

In [76]:
transaction_df.isnull().sum()

Customer_Company_Name         0
Customer_Sub_Region           0
First                         0
Last Deal                     0
L12M Transaction Count    48093
L12M Principal            48093
L12M Revenue              48093
L24M Transaction Count    23460
L24M Principal            23460
L24M Revenue              23460
L36M Transaction Count    19384
L36M Principal            19384
L36M Revenue              19384
dtype: int64

In [77]:
cols = transaction_df.columns[4:].tolist()

In [78]:
transaction_df.loc[transaction_df['Last Deal'] < '2018-10-01',cols] = 0

In [79]:
transaction_df.loc[transaction_df['Last Deal'] < '2019-10-01',cols[:6]] = 0

In [80]:
transaction_df.loc[transaction_df['Last Deal'] < '2020-10-01',cols[:3]] = 0

In [81]:
transaction_df.isnull().sum()

Customer_Company_Name         0
Customer_Sub_Region           0
First                         0
Last Deal                     0
L12M Transaction Count    15185
L12M Principal            15185
L12M Revenue              15185
L24M Transaction Count      197
L24M Principal              197
L24M Revenue                197
L36M Transaction Count      195
L36M Principal              195
L36M Revenue                195
dtype: int64

In [82]:
transaction_df = transaction_df[~transaction_df['L36M Revenue'].isnull()]

In [83]:
transaction_df.isnull().sum()

Customer_Company_Name         0
Customer_Sub_Region           0
First                         0
Last Deal                     0
L12M Transaction Count    15009
L12M Principal            15009
L12M Revenue              15009
L24M Transaction Count        4
L24M Principal                4
L24M Revenue                  4
L36M Transaction Count        0
L36M Principal                0
L36M Revenue                  0
dtype: int64

In [84]:
import fuzzymatcher

In [85]:
transaction_df.columns

Index(['Customer_Company_Name', 'Customer_Sub_Region', 'First', 'Last Deal',
       'L12M Transaction Count', 'L12M Principal', 'L12M Revenue',
       'L24M Transaction Count', 'L24M Principal', 'L24M Revenue',
       'L36M Transaction Count', 'L36M Principal', 'L36M Revenue'],
      dtype='object')

In [86]:
customer_df.columns

Index(['customer_id', 'customer_fiscal_start_date',
       'customer_last_txn_fiscal_date', 's_customer_region_grouped',
       'L12M Transaction Count', 'L24M Transaction Count',
       'L36M Transaction Count', 'L12M Principal', 'L24M Principal',
       'L36M Principal', 'L12M Revenue', 'L24M Revenue', 'L36M Revenue'],
      dtype='object')

In [87]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340784 entries, 0 to 340783
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_id                    340784 non-null  object        
 1   customer_fiscal_start_date     340784 non-null  datetime64[ns]
 2   customer_last_txn_fiscal_date  340784 non-null  datetime64[ns]
 3   s_customer_region_grouped      340784 non-null  object        
 4   L12M Transaction Count         340784 non-null  int64         
 5   L24M Transaction Count         340784 non-null  int64         
 6   L36M Transaction Count         340784 non-null  int64         
 7   L12M Principal                 340784 non-null  float64       
 8   L24M Principal                 340784 non-null  float64       
 9   L36M Principal                 340784 non-null  float64       
 10  L12M Revenue                   340784 non-null  float64       
 11  

In [88]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48728 entries, 0 to 48922
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Customer_Company_Name   48728 non-null  object        
 1   Customer_Sub_Region     48728 non-null  object        
 2   First                   48728 non-null  datetime64[ns]
 3   Last Deal               48728 non-null  datetime64[ns]
 4   L12M Transaction Count  33719 non-null  float64       
 5   L12M Principal          33719 non-null  float64       
 6   L12M Revenue            33719 non-null  float64       
 7   L24M Transaction Count  48724 non-null  float64       
 8   L24M Principal          48724 non-null  float64       
 9   L24M Revenue            48724 non-null  float64       
 10  L36M Transaction Count  48728 non-null  float64       
 11  L36M Principal          48728 non-null  float64       
 12  L36M Revenue            48728 non-null  float6

In [89]:
transaction_df['L12M Transaction Count'] = transaction_df['L12M Transaction Count'].fillna(0).astype(int)
transaction_df['L24M Transaction Count'] = transaction_df['L24M Transaction Count'].fillna(0).astype(int)
transaction_df['L36M Transaction Count'] = transaction_df['L36M Transaction Count'].fillna(0).astype(int)

In [90]:
transaction_df.isnull().sum()

Customer_Company_Name         0
Customer_Sub_Region           0
First                         0
Last Deal                     0
L12M Transaction Count        0
L12M Principal            15009
L12M Revenue              15009
L24M Transaction Count        0
L24M Principal                4
L24M Revenue                  4
L36M Transaction Count        0
L36M Principal                0
L36M Revenue                  0
dtype: int64

In [91]:
transaction_df.shape

(48728, 13)

In [92]:
transaction_Asia = transaction_df[transaction_df['Customer_Sub_Region'] == 'Asia']
customer_Asia = customer_df[customer_df['s_customer_region_grouped'] == 'Asia']

In [93]:
left_on = ['First', 'Last Deal','L12M Transaction Count', 'L12M Principal', 'L12M Revenue',
           'L24M Transaction Count', 'L24M Principal', 'L24M Revenue','L36M Transaction Count', 'L36M Principal',
           'L36M Revenue']

right_on = ['customer_fiscal_start_date','customer_last_txn_fiscal_date','L12M Transaction Count',
            'L12M Principal', 'L12M Revenue', 'L24M Transaction Count','L24M Principal' ,'L24M Revenue',
            'L36M Transaction Count','L36M Principal', 'L36M Revenue']

In [94]:
matched_results_Asia = fuzzymatcher.fuzzy_left_join(transaction_Asia,
                                            customer_Asia,
                                            left_on,
                                            right_on,
                                            left_id_col='Customer_Company_Name',
                                            right_id_col='customer_id')

In [95]:
pd.set_option('display.max_columns',30)
matched_results_Asia.sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,__id_left,__id_right,Customer_Company_Name,Customer_Sub_Region,First,Last Deal,L12M Transaction Count_left,L12M Principal_left,L12M Revenue_left,L24M Transaction Count_left,L24M Principal_left,L24M Revenue_left,L36M Transaction Count_left,L36M Principal_left,L36M Revenue_left,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped,L12M Transaction Count_right,L24M Transaction Count_right,L36M Transaction Count_right,L12M Principal_right,L24M Principal_right,L36M Principal_right,L12M Revenue_right,L24M Revenue_right,L36M Revenue_right
155667,1.057025,uschBriseEnterpitedLim,70AB7B87-9A70-4577-A498-F9AAF7BA8228,uschBriseEnterpitedLim,Asia,2019-10-07,2019-10-07,0,0.0,0.0,1,3314.683,68.035,1,3314.683,68.035,70AB7B87-9A70-4577-A498-F9AAF7BA8228,2020-07-07,2020-07-07,Asia,0,1,1,0.0,3314.683,3314.683,0.0,68.035,68.035
130275,1.014135,MojoeptsconcePtdlt,03EB6C57-2A70-4400-9C6A-F85ED6B551E6,MojoeptsconcePtdlt,Asia,2019-10-18,2019-12-13,0,0.0,0.0,5,38193.432,89.858,5,38193.432,89.858,03EB6C57-2A70-4400-9C6A-F85ED6B551E6,2020-07-18,2020-09-13,Asia,0,5,5,0.0,38193.432,38193.432,0.0,89.858,89.858
143679,1.002623,uavaGtiesAmeniePtdLt,8CA23DE0-A272-4772-9C29-06C4BC6A4B77,uavaGtiesAmeniePtdLt,Asia,2019-12-09,2019-12-09,0,0.0,0.0,1,12615.964,18.346,1,12615.964,18.346,8CA23DE0-A272-4772-9C29-06C4BC6A4B77,2020-09-09,2020-09-09,Asia,0,1,1,0.0,12615.964,12615.964,0.0,18.346,18.346
140543,0.975505,lkinWetemsSysitedLim,D6858A04-C7E2-4E36-B2B6-01F39F113D5F,lkinWetemsSysitedLim,Asia,2019-12-10,2019-12-10,0,0.0,0.0,1,16427.417,127.288,1,16427.417,127.288,D6858A04-C7E2-4E36-B2B6-01F39F113D5F,2020-10-09,2020-10-09,Asia,0,1,1,0.0,16427.417,16427.417,0.0,127.288,127.288
160000,0.913864,atanEilePtdLt,CAC82FF3-A1E4-49E8-8796-BDF9018047C3,atanEilePtdLt,Asia,2019-10-03,2019-10-09,0,0.0,0.0,3,1475.246,37.173,3,1475.246,37.173,CAC82FF3-A1E4-49E8-8796-BDF9018047C3,2020-03-07,2020-09-07,Asia,0,3,3,0.0,1475.246,1475.246,0.0,37.173,37.173


In [96]:
transaction_Europe = transaction_df[transaction_df['Customer_Sub_Region'] == 'Europe']
customer_Europe = customer_df[customer_df['s_customer_region_grouped'] == 'Europe']

In [97]:
matched_results_Europe = fuzzymatcher.fuzzy_left_join(transaction_Europe,
                                            customer_Europe,
                                            left_on,
                                            right_on,
                                            left_id_col='Customer_Company_Name',
                                            right_id_col='customer_id')

In [98]:
transaction_NA = transaction_df[transaction_df['Customer_Sub_Region'] == 'North America']
customer_NA = customer_df[customer_df['s_customer_region_grouped'] == 'North America']

In [99]:
matched_results_NA = fuzzymatcher.fuzzy_left_join(transaction_NA,
                                            customer_NA,
                                            left_on,
                                            right_on,
                                            left_id_col='Customer_Company_Name',
                                            right_id_col='customer_id')

In [100]:
transaction_ANZ = transaction_df[transaction_df['Customer_Sub_Region'] == 'ANZ']
customer_ANZ = customer_df[customer_df['s_customer_region_grouped'] == 'ANZ']

In [101]:
matched_results_ANZ = fuzzymatcher.fuzzy_left_join(transaction_ANZ,
                                            customer_ANZ,
                                            left_on,
                                            right_on,
                                            left_id_col='Customer_Company_Name',
                                            right_id_col='customer_id')

In [102]:
matched_results = pd.concat([matched_results_Asia,matched_results_Europe],axis=0)

In [103]:
matched_results = pd.concat([matched_results,matched_results_NA],axis=0)

In [104]:
matched_results = pd.concat([matched_results,matched_results_ANZ],axis=0)

In [105]:
matched_results.tail()

,best_match_score,__id_left,__id_right,Customer_Company_Name,Customer_Sub_Region,First,Last Deal,L12M Transaction Count_left,L12M Principal_left,L12M Revenue_left,L24M Transaction Count_left,L24M Principal_left,L24M Revenue_left,L36M Transaction Count_left,L36M Principal_left,L36M Revenue_left,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped,L12M Transaction Count_right,L24M Transaction Count_right,L36M Transaction Count_right,L12M Principal_right,L24M Principal_right,L36M Principal_right,L12M Revenue_right,L24M Revenue_right,L36M Revenue_right
4298109,-0.427759,OACMYPTDLT,9F40244D-C598-4C21-AADD-89B007523140,OACMYPTDLT,ANZ,2020-10-15,2020-10-15,0,NaN,NaN,1,50.00,0.000,1,50.00,0.000,9F40244D-C598-4C21-AADD-89B007523140,2016-10-15,2017-12-09,ANZ,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4298316,-1.097425,weetSssomBloandyCyPtitedLim,9F40244D-C598-4C21-AADD-89B007523140,weetSssomBloandyCyPtitedLim,ANZ,2020-12-09,2020-12-09,0,NaN,NaN,1,47.50,16.078,1,47.50,16.078,9F40244D-C598-4C21-AADD-89B007523140,2016-10-15,2017-12-09,ANZ,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4298567,0.257643,DaPoyPtdLt,D5ED2B22-D6F9-413B-8CFB-2B14F5EE308E,DaPoyPtdLt,ANZ,2013-03-05,2013-03-05,0,0.0,0.0,0,0.00,0.000,0,0.00,0.000,D5ED2B22-D6F9-413B-8CFB-2B14F5EE308E,2013-03-07,2017-03-05,ANZ,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4298617,-0.352704,edinRubbyPtdLt,E36AFA27-BC80-4E56-92EB-D960D2785ED2,edinRubbyPtdLt,ANZ,2018-10-10,2018-10-10,0,0.0,0.0,0,0.00,0.000,1,21.50,15.859,E36AFA27-BC80-4E56-92EB-D960D2785ED2,2011-10-31,2017-10-10,ANZ,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4298828,-1.006080,OXoolsTAUyPtdLtAT/ITECUNPPLYSUPANYCOM,2E8A90F4-CD70-4FFB-9142-E5292DFB96B8,OXoolsTAUyPtdLtAT/ITECUNPPLYSUPANYCOM,ANZ,2020-09-17,2020-09-17,0,0.0,0.0,1,9.91,15.404,1,9.91,15.404,2E8A90F4-CD70-4FFB-9142-E5292DFB96B8,2015-09-17,2017-01-27,ANZ,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
final = matched_results[matched_results['best_match_score']>=0.6]

In [146]:
final['customer_id'].nunique()

347

In [108]:
file_name = 'Matched_results.xlsx'
matched_results.to_excel(file_name)

In [109]:
matched_results.shape

(48728, 29)

In [110]:
matched_results['customer_id'].nunique()

20252

In [147]:
final.shape

(347, 29)

In [124]:
final1 = matched_results[matched_results['best_match_score']>=0.2]

In [121]:
matched_results[matched_results['best_match_score']>=0.2]['customer_id'].nunique()

10323

In [125]:
final1.shape

(13431, 29)

In [119]:
matched_results.sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,__id_left,__id_right,Customer_Company_Name,Customer_Sub_Region,First,Last Deal,L12M Transaction Count_left,L12M Principal_left,L12M Revenue_left,L24M Transaction Count_left,L24M Principal_left,L24M Revenue_left,L36M Transaction Count_left,L36M Principal_left,L36M Revenue_left,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped,L12M Transaction Count_right,L24M Transaction Count_right,L36M Transaction Count_right,L12M Principal_right,L24M Principal_right,L36M Principal_right,L12M Revenue_right,L24M Revenue_right,L36M Revenue_right
2338824,1.206367,maryPrialesSaliaAustryPtdLt,ED43E885-1A33-496D-83C3-43AD385F7716,maryPrialesSaliaAustryPtdLt,ANZ,2018-07-27,2019-12-17,0,0.0,0.0,10,273119.310,2440.760,11,290413.220,2602.259,ED43E885-1A33-496D-83C3-43AD385F7716,2019-04-27,2020-09-17,ANZ,0,10,11,0.0,273119.310,290413.220,0.0,2440.760,2602.259
2049849,1.202743,landHylebyPtdLt,C2C62499-F2FE-47BD-AC42-74A9FB5827C1,landHylebyPtdLt,ANZ,2006-03-16,2019-10-02,0,0.0,0.0,8,69607.760,420.202,8,69607.760,420.202,C2C62499-F2FE-47BD-AC42-74A9FB5827C1,2006-12-16,2020-02-07,ANZ,0,8,8,0.0,69607.760,69607.760,0.0,420.202,420.202
2538742,1.199702,AWICGYPTDLT,83F9AD37-C4A3-49EB-BA7D-7973360DC022,AWICGYPTDLT,ANZ,2019-08-06,2019-12-09,0,0.0,0.0,2,224848.444,1286.374,2,224848.444,1286.374,83F9AD37-C4A3-49EB-BA7D-7973360DC022,2020-06-05,2020-09-09,ANZ,0,2,2,0.0,224848.444,224848.444,0.0,1286.374,1286.374
370934,1.180579,ESTASIONALINTERNATIDAYSHOLIITEDLIM,09382456-34AB-4320-A31C-2D5DE4920C2F,ESTASIONALINTERNATIDAYSHOLIITEDLIM,Europe,2019-07-24,2019-12-16,0,0.0,0.0,5,700104.372,2260.950,5,700104.372,2260.950,09382456-34AB-4320-A31C-2D5DE4920C2F,2020-04-24,2020-09-16,Europe,0,5,5,0.0,700104.372,700104.372,0.0,2260.950,2260.950
181202,1.160322,AKMDuperSyPtdLt,5F9E15B9-B9BB-4986-9715-8272B8EE5ED4,AKMDuperSyPtdLt,ANZ,2019-09-17,2019-10-16,0,0.0,0.0,2,507233.888,2199.200,2,507233.888,2199.200,5F9E15B9-B9BB-4986-9715-8272B8EE5ED4,2020-06-17,2020-07-16,ANZ,0,2,2,0.0,507233.888,507233.888,0.0,2199.200,2199.200


In [120]:
matched_results.sort_values(by=['best_match_score'], ascending=False).tail()

,best_match_score,__id_left,__id_right,Customer_Company_Name,Customer_Sub_Region,First,Last Deal,L12M Transaction Count_left,L12M Principal_left,L12M Revenue_left,L24M Transaction Count_left,L24M Principal_left,L24M Revenue_left,L36M Transaction Count_left,L36M Principal_left,L36M Revenue_left,customer_id,customer_fiscal_start_date,customer_last_txn_fiscal_date,s_customer_region_grouped,L12M Transaction Count_right,L24M Transaction Count_right,L36M Transaction Count_right,L12M Principal_right,L24M Principal_right,L36M Principal_right,L12M Revenue_right,L24M Revenue_right,L36M Revenue_right
20360,-3.411469,RPAKPOWEROUPGYPTDLT,D850E033-2361-4401-B5E8-A87D4874BB2F,RPAKPOWEROUPGYPTDLT,ANZ,2013-06-10,2021-02-09,313,9.755590e+06,8418.789,612,1.720454e+07,13367.351,885,2.302472e+07,17623.700,D850E033-2361-4401-B5E8-A87D4874BB2F,2014-06-10,2021-09-22,ANZ,1,2,5,3000.000,6000.000,13800.000,70.032,140.109,324.562
202585,-3.453193,MdanHALERWHOLESCOYPTDLT,4E3C72E4-057A-4B13-817A-ACB81D14600D,MdanHALERWHOLESCOYPTDLT,ANZ,2019-07-11,2021-02-09,49,1.054990e+06,2157.704,77,1.436925e+06,2922.935,77,1.436925e+06,2922.935,4E3C72E4-057A-4B13-817A-ACB81D14600D,2019-07-03,2021-09-20,ANZ,1,1,3,600.000,600.000,2050.000,14.992,14.992,44.013
20102,-3.520336,A.R.B.roupGyPtdLt,D7F72E0C-AC38-4A72-AA67-F2FCB64E3749,A.R.B.roupGyPtdLt,ANZ,2009-09-24,2021-02-08,415,1.501833e+07,54254.326,724,2.549274e+07,85426.845,1051,3.625500e+07,117435.198,D7F72E0C-AC38-4A72-AA67-F2FCB64E3749,2010-02-09,2021-10-08,ANZ,2,3,3,50000.000,100000.000,100000.000,599.494,1243.500,1243.500
72854,-3.534721,lAlightBrdingBuilialsMateryPtdLt,33F4586D-B5FD-4E1C-B2A2-4BE106E8EA28,lAlightBrdingBuilialsMateryPtdLt,ANZ,2014-11-13,2021-02-09,103,2.135161e+06,12218.831,162,3.668373e+06,19468.678,252,6.202132e+06,32387.363,33F4586D-B5FD-4E1C-B2A2-4BE106E8EA28,2014-06-11,2021-02-09,ANZ,4,5,5,39600.000,49100.000,49100.000,594.295,738.205,738.205
22028,-3.594366,DaletersImporust)(AyPtdLt,95E1C2F1-A590-48A4-86C6-5485C0D5EF9A,DaletersImporust)(AyPtdLt,ANZ,2006-08-09,2021-02-01,74,1.284536e+06,5639.490,146,2.465596e+06,8130.903,230,3.794225e+06,10435.009,95E1C2F1-A590-48A4-86C6-5485C0D5EF9A,2010-09-08,2021-01-02,ANZ,1,1,1,5504.666,5504.666,5504.666,52.187,52.187,52.187
